In [ ]:
# Import necessary libraries
import pandas as pd          # For data manipulation and handling
import numpy as np           # For numerical operations
from sklearn.model_selection import train_test_split  # For splitting the data into training and testing sets
from sklearn.ensemble import RandomForestClassifier   # For the machine learning model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # For evaluating the model


In [ ]:
# Load the data
data = pd.read_csv('US_Accidents_March23_sampled_500k.csv')

# Filter the data for rows where the City is "Los Angeles"
data = data[data['City'] == 'Los Angeles']

# Display the first few rows of the filtered data
# print(la_data.head())
# Display only the "City" column for rows where the City is "Los Angeles"
# print(la_data['City'])



In [ ]:
# Drop the specified columns
columns_to_drop = ['Description', 'End_Lng', 'End_Lat', 'Astronomical_Twilight', 
                   'Nautical_Twilight', 'Civil_Twilight', 'Country', 'Timezone', 
                   'Airport_Code', 'County', 'State', 'City']
data = data.drop(columns=columns_to_drop)

# Verify the changes
print(data.head())



In [ ]:
# Convert the 'Start_Time' column to datetime, setting invalid parsing as NaT
data['Start_Time'] = pd.to_datetime(data['Start_Time'], errors='coerce', format='%Y-%m-%d %H:%M')

# Drop rows where 'Start_Time' is NaT (invalid date format)
data = data.dropna(subset=['Start_Time'])

# Verify the changes by displaying the first few rows
print(data.head())


In [ ]:
# Get the number of rows in the data
num_rows = data.shape[0]
print("Number of rows:", num_rows)


In [ ]:
# Check the number of missing values in each column before filling or dropping
print("Missing values in each column before filling:")
print(data.isnull().sum())


In [ ]:
# Drop the specified columns
columns_to_drop = ['Precipitation(in)', 'Wind_Chill(F)', 'Wind_Direction', 'Wind_Speed(mph)']
data = data.drop(columns=columns_to_drop)

# Verify the changes
# print(data.head())
print(data.columns)


In [ ]:
# Decide to remove some columns again
columns_to_drop = ['ID', 'End_Time', 'Distance(mi)', 'Street', 'Zipcode', 'Weather_Timestamp','Pressure(in)','Source']
data = data.drop(columns=columns_to_drop)

# Verify the changes
# print(data.head())
print(data.columns)

In [ ]:
# Define the columns to replace True/False with 1/0
boolean_columns = [
    'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 
    'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 
    'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop'
]

# Replace True/False with 1/0 for the specified columns
data[boolean_columns] = data[boolean_columns].astype(int)

# Verify the changes
print(data[boolean_columns].head())


In [ ]:
# Encode 'Sunrise_Sunset' column: 1 for 'Day', 0 for 'Night'
data['Sunrise_Sunset'] = data['Sunrise_Sunset'].map({'Day': 1, 'Night': 0})

# Verify the changes
print(data['Sunrise_Sunset'].head())


In [ ]:
# Define the mapping for weather conditions
weather_mapping = {
    'Clear': 1, 'Fair': 1,
    'Cloudy': 2, 'Partly Cloudy': 2, 'Mostly Cloudy': 2, 'Overcast': 2,
    'Cloudy / Windy': 2, 'Mostly Cloudy / Windy': 2,
    'Light Rain': 3, 'Rain': 3, 'Heavy Rain': 3, 'Rain / Windy': 3, 'Light Rain with Thunder': 3,
    'Thunder': 4, 'T-Storm': 4, 'Heavy T-Storm': 4,
    'Fog': 5, 'Patches of Fog': 5, 'Mist': 5, 'Haze': 5,
    'Smoke': 0, 'Scattered Clouds': 0, None: 0, '': 0  # Handle blanks and unknowns
}

# Apply the mapping to the 'Weather_Condition' column
data['Weather_Condition'] = data['Weather_Condition'].map(weather_mapping)

# Verify the changes
print(data['Weather_Condition'].unique())


In [ ]:
# Check the number of missing values in each column before filling or dropping
print("Missing values in each column before filling:")
print(data.isnull().sum())

In [ ]:
# Remove rows with missing values
data = data.dropna()

# Verify the changes
print("Number of missing values in each column after removing rows:")
print(data.isnull().sum())


In [ ]:
# Get the number of rows in the data
num_rows = data.shape[0]
print("Number of rows:", num_rows)

In [ ]:
# Ensure the "Start_Time" column is in datetime format
data['Start_Time'] = pd.to_datetime(data['Start_Time'], errors='coerce')

# Sort the data by "Start_Time" in ascending order (oldest to newest)
data = data.sort_values(by='Start_Time', ascending=True).reset_index(drop=True)

# Display the first few rows to verify the sorting
print(data.head())
# Save the data into an Excel file named "x_features.xlsx"
output_file = "x_features.xlsx"
data.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")


In [ ]:
print(data.columns)

In [ ]:

from sklearn.cluster import KMeans
from scipy.spatial import ConvexHull
import folium
# Step 1: Clustering the points
coords = data[['Start_Lat', 'Start_Lng']].values
n_clusters = 10  # Adjust the number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
data['Cluster'] = kmeans.fit_predict(coords)

# Step 2: Create Convex Hulls for each cluster
cluster_polygons = []
for cluster in range(n_clusters):
    cluster_points = coords[data['Cluster'] == cluster]
    if len(cluster_points) >= 3:  # ConvexHull requires at least 3 points
        hull = ConvexHull(cluster_points)
        polygon = [cluster_points[vertex] for vertex in hull.vertices]
        cluster_polygons.append((cluster, polygon))

# Step 3: Visualize using Folium
map_la = folium.Map(location=[34.05, -118.25], zoom_start=10)

# Add polygons to the map
for cluster, polygon in cluster_polygons:
    folium.Polygon(
        locations=polygon,
        color=f"#{(cluster * 10000):06x}",
        fill=True,
        fill_opacity=0.4,
        popup=f"Cluster {cluster}"
    ).add_to(map_la)

# Add points to the map
for _, row in data.iterrows():
    folium.CircleMarker(
        location=[row['Start_Lat'], row['Start_Lng']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(map_la)

# Save the map to an HTML file
map_la.save("la_regions_map.html")
print("Map with regions saved as 'la_regions_map.html'. Open this file in a browser.")



In [ ]:
# Count the number of clusters
n_clusters = data['Cluster'].nunique()
print(f"Number of clusters: {n_clusters}")



In [ ]:

from datetime import datetime, timedelta

# Initialize an empty DataFrame to store the output
all_years_output = pd.DataFrame()

# Define start date and end date
start_date = datetime(2016, 3, 22)
end_date = datetime(2023, 3, 31)

# Iterate through all dates in the specified range
current_date = start_date
while current_date <= end_date:
    # Filter data for the specific date
    filtered_data = data[data['Start_Time'].dt.date == current_date.date()]
    
    # Count the total number of accidents (rows) for that date
    total_accidents_for_day = len(filtered_data)
    
    # Count the number of accidents in each cluster
    accidents_per_cluster = filtered_data['Cluster'].value_counts()
    
    # Include clusters with 0 accidents
    all_clusters = range(10)  # Assuming 10 clusters
    accidents_per_cluster = accidents_per_cluster.reindex(all_clusters, fill_value=0)
    
    # Calculate the total number of accidents on that date
    total_accidents = accidents_per_cluster.sum()
    
    # Calculate the probability of accidents for each cluster
    probability_per_cluster = accidents_per_cluster / total_accidents if total_accidents > 0 else accidents_per_cluster
    
    # Prepare the row for this specific date
    row = {'Date': current_date.date()}
    for cluster in all_clusters:
        row[f'Cluster_{cluster}'] = probability_per_cluster[cluster]
    row['Total_Accidents'] = total_accidents_for_day
    
    # Append the row to the output DataFrame
    all_years_output = pd.concat([all_years_output, pd.DataFrame([row])], ignore_index=True)
    
    # Increment the date by one day
    current_date += timedelta(days=1)

# Save the output to an Excel file
output_file = "accident_data_2016_2023_subset.xlsx"
all_years_output.to_excel(output_file, index=False)

print(f"Accident data from {start_date.date()} to {end_date.date()} has been saved as '{output_file}'.")


In [ ]:
# Filter the data for the specific date
specific_date = "2016-03-23"
filtered_data = data[data['Start_Time'].dt.date == pd.to_datetime(specific_date).date()]

# Display the filtered data with the specified columns
columns_to_display = [
    'Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Temperature(F)',
    'Humidity(%)', 'Visibility(mi)', 'Weather_Condition', 'Amenity', 'Bump',
    'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout',
    'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
    'Sunrise_Sunset'
]

filtered_data = filtered_data[columns_to_display]

# Display the data
print(filtered_data)


In [ ]:

from sklearn.cluster import KMeans

# Assuming 'data' is your DataFrame with binary columns and latitude/longitude
binary_columns = [
    "Weather_Condition", "Amenity", "Bump", "Crossing", "Give_Way", 
    "Junction", "No_Exit", "Railway", "Roundabout", "Station", 
    "Stop", "Traffic_Calming", "Traffic_Signal", "Turning_Loop", 
    "Sunrise_Sunset"
]

# Step 1: Perform clustering
coords = data[['Start_Lat', 'Start_Lng']].values
n_clusters = 10  # Adjust this if needed
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
data['Cluster'] = kmeans.fit_predict(coords)

# Step 2: Create new cluster-specific columns
for col in binary_columns:
    for cluster in range(n_clusters):
        new_col_name = f"{col}_Cluster{cluster}"
        # Assign 1 to the respective cluster column if the cluster matches and the binary column value is 1
        data[new_col_name] = np.where(data['Cluster'] == cluster, data[col], 0)

# Drop the original 'Cluster' column if no longer needed
# data = data.drop(columns=['Cluster'])

# Save to Excel for review
data.to_excel("x_features_clustered.xlsx", index=False)

print("New cluster-specific columns added and saved to 'x_features_clustered.xlsx'.")


In [ ]:
# # Specific point to check
# specific_point = np.array([[34.018902, -118.173264]])

# # Predict the cluster for the specific point
# predicted_cluster = kmeans.predict(specific_point)[0]

# print(f"The point {specific_point[0]} belongs to Cluster {predicted_cluster}.")


In [ ]:


# Load the data from the provided file
file_name = "x_features_clustered.xlsx"
data = pd.read_excel(file_name)

# Select only the desired columns
columns_to_keep = ['Start_Time', 'Weather_Condition', 'Cluster']
filtered_data = data[columns_to_keep]

# Save the filtered data to a new file
output_file_name = "Weather_Condition.xlsx"
filtered_data.to_excel(output_file_name, index=False)

# import ace_tools as tools; tools.display_dataframe_to_user(name="Filtered Weather Condition Data", dataframe=filtered_data)

In [ ]:


# Load the Weather_Condition file
weather_data = pd.read_excel("Weather_Condition.xlsx")

# Create new columns for each cluster
n_clusters = 10  # Assuming 10 clusters
for cluster in range(n_clusters):
    cluster_col_name = f"Weather_Condition_Cluster{cluster}"
    # Assign weather condition to the corresponding cluster column
    weather_data[cluster_col_name] = weather_data.apply(
        lambda row: row["Weather_Condition"] if row["Cluster"] == cluster else 0, axis=1
    )

# Save the updated data back to the same file
weather_data.to_excel("Weather_Condition.xlsx", index=False)

# import ace_tools as tools; tools.display_dataframe_to_user(name="Weather Condition with Cluster Columns", dataframe=weather_data)


In [ ]:


# Read the Excel file
file_path = "Weather_Condition.xlsx"  # Replace with the correct file path
df = pd.read_excel(file_path)

# Ensure 'Start_Time' is in datetime format
df["Start_Time"] = pd.to_datetime(df["Start_Time"])

# Extract the date from Start_Time to group by date
df["Date"] = df["Start_Time"].dt.date

# Group by Date and Cluster to count the number of points and aggregate weather conditions
summary = (
    df.groupby(["Date", "Cluster"])
    .agg(
        Points_Count=("Weather_Condition", "size"),  # Count the number of points
        Weather_Condition_Mode=("Weather_Condition", lambda x: x.mode()[0] if not x.mode().empty else None),  # Calculate the mode
    )
    .reset_index()
)

# Optionally, save the summary to a new Excel file
summary.to_excel("Cluster_Weather_Condition_Summary.xlsx", index=False)

# Display the result
# import ace_tools as tools; tools.display_dataframe_to_user(name="Cluster Weather Condition Summary", dataframe=summary)


In [ ]:


# Load your summarized data
summary_file = "Cluster_Weather_Condition_Summary.xlsx"  # Replace with your file path
summary = pd.read_excel(summary_file)

# Create a list of all unique cluster column names
clusters = [f"Weather_Condition_Cluster{i}" for i in range(10)]

# Initialize an empty DataFrame for the final result
final_data = pd.DataFrame(columns=["Date"] + clusters)

# Iterate through each date
for date, group in summary.groupby("Date"):
    # Create a row with all clusters initialized to 0
    row = {cluster: 0 for cluster in clusters}
    row["Date"] = date
    
    # Populate the mode for each cluster
    for _, row_data in group.iterrows():
        cluster_name = f"Weather_Condition_Cluster{int(row_data['Cluster'])}"
        row[cluster_name] = row_data["Weather_Condition_Mode"]
    
    # Append the row to the final DataFrame
    final_data = pd.concat([final_data, pd.DataFrame([row])], ignore_index=True)

# Save the final DataFrame to a new Excel file
final_data.to_excel("Aggregated_Weather_Conditions_By_Date.xlsx", index=False)

# Display the result
# import ace_tools as tools; tools.display_dataframe_to_user(name="Aggregated Weather Conditions by Date", dataframe=final_data)


In [ ]:


# Load the data from the provided file
file_name = "x_features_clustered.xlsx"  
data = pd.read_excel(file_name)

# Select only the desired columns
columns_to_keep = ['Start_Time', 'Severity', 'Cluster']  
filtered_data = data[columns_to_keep]

# Save the filtered data to a new file
output_file_name = "Severity_Condition.xlsx"
filtered_data.to_excel(output_file_name, index=False)

# Ensure 'Start_Time' is in datetime format
filtered_data["Start_Time"] = pd.to_datetime(filtered_data["Start_Time"])

# Extract the date from Start_Time to group by date
filtered_data["Date"] = filtered_data["Start_Time"].dt.date

# Group by Date and Cluster to count the number of points and aggregate severity
summary = (
    filtered_data.groupby(["Date", "Cluster"])
    .agg(
        Points_Count=("Severity", "size"),  # Count the number of points
        Severity_Mode=("Severity", lambda x: x.mode()[0] if not x.mode().empty else None),  # Calculate the mode of Severity
    )
    .reset_index()
)

# Save the summary to a new Excel file
summary_file = "Cluster_Severity_Summary.xlsx"
summary.to_excel(summary_file, index=False)

# Create a list of all unique cluster column names
clusters = [f"Severity_Cluster{i}" for i in range(10)]

# Initialize an empty DataFrame for the final result
final_data = pd.DataFrame(columns=["Date"] + clusters)

# Iterate through each date
for date, group in summary.groupby("Date"):
    # Create a row with all clusters initialized to 0
    row = {cluster: 0 for cluster in clusters}
    row["Date"] = date
    
    # Populate the mode for each cluster
    for _, row_data in group.iterrows():
        cluster_name = f"Severity_Cluster{int(row_data['Cluster'])}"
        row[cluster_name] = row_data["Severity_Mode"]
    
    # Append the row to the final DataFrame
    final_data = pd.concat([final_data, pd.DataFrame([row])], ignore_index=True)

# Save the final DataFrame to a new Excel file
final_data.to_excel("Aggregated_Severity_By_Date.xlsx", index=False)

# Display the result
# import ace_tools as tools; tools.display_dataframe_to_user(name="Aggregated Severity by Date", dataframe=final_data)


In [ ]:

# Load the Excel file
file_name = "x_features_clustered.xlsx"  # Replace with the actual file path
data = pd.read_excel(file_name)

# Drop the "Severity" and "Weather_Condition" columns, and all "Weather_Condition_ClusterX" columns
columns_to_drop = ['Severity', 'Weather_Condition','cluster'] + [f"Weather_Condition_Cluster{i}" for i in range(10)]
data = data.drop(columns=columns_to_drop, errors='ignore')

# Save the updated data to a new Excel file
output_file_name = "x_features_clustered_cleaned.xlsx"
data.to_excel(output_file_name, index=False)

# Display the result
# import ace_tools as tools; tools.display_dataframe_to_user(name="Cleaned Features Clustered Data", dataframe=data)


In [ ]:

from datetime import datetime, timedelta

# Load the data
file_name = "x_features_clustered_cleaned.xlsx"  # Replace with the correct file name
data = pd.read_excel(file_name)

# Ensure the 'Start_Time' column is in datetime format
data['Start_Time'] = pd.to_datetime(data['Start_Time'])

# Define the start and end dates
start_date = datetime(2016, 3, 22)
end_date = datetime(2023, 3, 31)

# Prepare a list to store aggregated rows
aggregated_rows = []

# Loop through each day in the range
current_date = start_date
while current_date <= end_date:
    # Filter the data for the specific date
    filtered_data = data[data['Start_Time'].dt.date == current_date.date()]
    
    # Skip if there's no data for the current date
    if filtered_data.empty:
        current_date += timedelta(days=1)
        continue
    
    # Initialize aggregated data for the specific date
    aggregated_row = {'Date': current_date.date()}
    
    # Step 1: Calculate mean for specific numeric columns
    numeric_columns = ['Temperature(F)', 'Humidity(%)', 'Visibility(mi)']  # Replace with actual numeric columns
    for col in numeric_columns:
        aggregated_row[col] = filtered_data[col].mean()
    
    # Step 2: Process binary and categorical columns by cluster
    binary_columns = [
        'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
        'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 
        'Turning_Loop', 'Sunrise_Sunset'  # Replace with actual binary columns
    ]
    n_clusters = 10  # Assuming 10 clusters
    for cluster in range(n_clusters):
        cluster_data = filtered_data[filtered_data['Cluster'] == cluster]
        
        # Process binary columns
        for col in binary_columns:
            column_name = f"{col}_Cluster{cluster}"
            if not cluster_data.empty:
                # Calculate the mean for binary columns (probability)
                aggregated_row[column_name] = cluster_data[col].mean()
            else:
                aggregated_row[column_name] = 0  # Default to 0 if no points in cluster
    
    # Append the aggregated row for the current date
    aggregated_rows.append(aggregated_row)
    
    # Move to the next day
    current_date += timedelta(days=1)

# Convert all aggregated rows into a DataFrame
aggregated_df = pd.DataFrame(aggregated_rows)

# Save the result to an Excel file
output_file_name = "aggregated_data_2016_to_2023.xlsx"
aggregated_df.to_excel(output_file_name, index=False)

print(f"Aggregated data from 2016-03-22 to 2023-03-31 has been saved to '{output_file_name}'.")










In [ ]:



# Load the three files
weather_file = "Aggregated_Weather_Conditions_By_Date.xlsx"
severity_file = "Aggregated_Severity_By_Date.xlsx"
aggregated_file = "aggregated_data_2016_to_2023.xlsx"

weather_data = pd.read_excel(weather_file)
severity_data = pd.read_excel(severity_file)
aggregated_data = pd.read_excel(aggregated_file)

# Drop the "Date" column from weather_data and severity_data
weather_data = weather_data.drop(columns=['Date'], errors='ignore')
severity_data = severity_data.drop(columns=['Date'], errors='ignore')

# Combine the columns from weather_data and severity_data into aggregated_data
final_data = pd.concat([aggregated_data, weather_data, severity_data], axis=1)

# Save the final result to a new Excel file
output_file_name = "final_aggregated_data.xlsx"
final_data.to_excel(output_file_name, index=False)

print(f"The final aggregated data has been saved to '{output_file_name}'.")


In [ ]:


# Load the Excel file
file_name = "final_aggregated_data.xlsx"  # Replace with the correct file name
data = pd.read_excel(file_name)

# Remove the time part from the Date column
data["Date"] = pd.to_datetime(data["Date"]).dt.date

# Save the updated data to a new file
output_file_name = "Final_X_Features.xlsx"
data.to_excel(output_file_name, index=False)

print(f"The updated file has been saved as '{output_file_name}'.")


In [ ]:


# File names
file1 = "Final_X_Features.xlsx"
file2 = "accident_data_2016_2023_subset.xlsx"

# Load the files
data1 = pd.read_excel(file1)
data2 = pd.read_excel(file2)

# Get the number of rows for each file
rows_file1 = len(data1)
rows_file2 = len(data2)

# Display the results
print(f"Number of rows in '{file1}': {rows_file1}")
print(f"Number of rows in '{file2}': {rows_file2}")


In [ ]:


# File names
file1 = "Final_X_Features.xlsx"
file2 = "accident_data_2016_2023_subset.xlsx"

# Load the files
data1 = pd.read_excel(file1)
data2 = pd.read_excel(file2)

# Ensure 'Date' columns are in the same format
data1["Date"] = pd.to_datetime(data1["Date"]).dt.date
data2["Date"] = pd.to_datetime(data2["Date"]).dt.date

# Identify dates present in one file but not the other
dates_in_file1_not_in_file2 = set(data1["Date"]) - set(data2["Date"])
dates_in_file2_not_in_file1 = set(data2["Date"]) - set(data1["Date"])

# Display the mismatched dates
if dates_in_file1_not_in_file2:
    print("Dates in Final_X_Features but not in accident_data_2016_2023_subset:")
    print(dates_in_file1_not_in_file2)

if dates_in_file2_not_in_file1:
    print("Dates in accident_data_2016_2023_subset but not in Final_X_Features:")
    print(dates_in_file2_not_in_file1)

# Optionally save mismatched dates to files
mismatched_dates = {
    "Dates in Final_X_Features but not in accident_data_2016_2023_subset": list(dates_in_file1_not_in_file2),
    "Dates in accident_data_2016_2023_subset but not in Final_X_Features": list(dates_in_file2_not_in_file1),
}

output_file_name = "mismatched_dates.xlsx"
with pd.ExcelWriter(output_file_name) as writer:
    pd.DataFrame({"Mismatched Dates": list(dates_in_file1_not_in_file2)}).to_excel(
        writer, sheet_name="Dates_in_Final_X_Features", index=False
    )
    pd.DataFrame({"Mismatched Dates": list(dates_in_file2_not_in_file1)}).to_excel(
        writer, sheet_name="Dates_in_accident_data", index=False
    )

print(f"Mismatched dates have been saved to '{output_file_name}'.")


In [ ]:

# Load the file
file_name = "accident_data_2016_2023_subset.xlsx"
data = pd.read_excel(file_name)

# Drop rows where all numeric columns are zero
# Only consider numeric columns for the all-zero check
numeric_columns = data.select_dtypes(include=['number']).columns
cleaned_data = data.loc[~(data[numeric_columns] == 0).all(axis=1)]

# Save the cleaned data to a new file
output_file_name = "Final_Y.xlsx"
cleaned_data.to_excel(output_file_name, index=False)

print(f"The cleaned file has been saved as '{output_file_name}'.")


In [ ]:


# Load the cleaned file
file_name = "Final_Y.xlsx"
data = pd.read_excel(file_name)

# Get the number of rows
number_of_rows = len(data)

print(f"The number of rows in '{file_name}' is: {number_of_rows}")


In [ ]:


# Load the file
file_name = "Final_Y.xlsx"
data = pd.read_excel(file_name)

# Remove the time part from the Date column
data["Date"] = pd.to_datetime(data["Date"]).dt.date

# Save the updated data to a temporary file
output_file_name = "Final_Y.xlsx"
data.to_excel(output_file_name, index=False)

print(f"The updated file has been saved as '{output_file_name}'.")



In [ ]:
import pandas as pd
from sklearn.utils import shuffle

# Load the files
x_file = "Final_X_Features.xlsx"
y_file = "Final_Y.xlsx"

X = pd.read_excel(x_file)
Y = pd.read_excel(y_file)

# Ensure the rows are aligned by 'Date' if necessary
if "Date" in X.columns and "Date" in Y.columns:
    X = X.sort_values(by="Date").reset_index(drop=True)
    Y = Y.sort_values(by="Date").reset_index(drop=True)

# Shuffle the rows while keeping X and Y aligned
X, Y = shuffle(X, Y, random_state=42)

# Save the shuffled data back to files for later use
X.to_excel("Shuffled_X_Features.xlsx", index=False)
Y.to_excel("Shuffled_Y.xlsx", index=False)

print("Shuffled X and Y have been saved as 'Shuffled_X_Features.xlsx' and 'Shuffled_Y.xlsx'.")


In [ ]:
# (Building the model)
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

print("Libraries imported successfully!")


In [ ]:
# Load the shuffled datasets
X_file = "Shuffled_X_Features.xlsx"
Y_file = "Shuffled_Y.xlsx"

X = pd.read_excel(X_file)
y = pd.read_excel(Y_file)

# Drop the 'Date' column from both X and y, if it exists
if "Date" in X.columns:
    X = X.drop(columns=["Date"])
if "Date" in y.columns:
    y = y.drop(columns=["Date"])

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data successfully split into training and testing sets.")
print(f"Training set size: {X_train.shape[0]} rows")
print(f"Testing set size: {X_test.shape[0]} rows")


In [ ]:
# Initialize the model
model = RandomForestRegressor(random_state=42)

# Train the model
model.fit(X_train, y_train)

print("Model training completed!")


In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
r2 = r2_score(y_test, y_pred)  # R-squared score

# Display the evaluation results
print(f"Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 Score): {r2}")


In [ ]:


# Ensure y_test and y_pred are DataFrames for saving
y_test_df = pd.DataFrame(y_test).reset_index(drop=True)
y_pred_df = pd.DataFrame(y_pred, columns=y_test_df.columns).reset_index(drop=True)

# Save to Excel files
y_test_file = "y_test.xlsx"
y_pred_file = "y_pred.xlsx"

y_test_df.to_excel(y_test_file, index=False)
y_pred_df.to_excel(y_pred_file, index=False)

print(f"y_test has been saved to '{y_test_file}'.")
print(f"y_pred has been saved to '{y_pred_file}'.")


In [ ]:


# Create a scatter plot to visualize the predictions vs. actual values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.6, edgecolor='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', linewidth=2)
plt.title("Predictions vs. Actual Values", fontsize=14)
plt.xlabel("Actual Values (y_test)", fontsize=12)
plt.ylabel("Predicted Values (y_pred)", fontsize=12)
plt.grid(alpha=0.4)
plt.show()


In [ ]:

# Ensure y_test and y_pred are flattened and of the same size
y_test_flat = np.array(y_test).flatten()  # Convert to a flat array
y_pred_flat = np.array(y_pred).flatten()  # Convert to a flat array

# Check that sizes match
if len(y_test_flat) != len(y_pred_flat):
    print("Error: y_test and y_pred must have the same size.")
else:
    # Create a scatter plot for actual vs predicted values with different colors
    plt.figure(figsize=(10, 6))
    plt.scatter(np.arange(len(y_test_flat)), y_test_flat, color='blue', label='Actual Values', alpha=0.6)
    plt.scatter(np.arange(len(y_pred_flat)), y_pred_flat, color='orange', label='Predicted Values', alpha=0.6)
    plt.title("Actual vs Predicted Values", fontsize=14)
    plt.xlabel("Index", fontsize=12)
    plt.ylabel("Values", fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(alpha=0.4)
    plt.show()


In [ ]:

# Extract feature importance from the trained model
feature_importances = model.feature_importances_

# Match the importances with feature names
feature_names = X_train.columns
importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": feature_importances
}).sort_values(by="Importance", ascending=False)

# Display the top features
print("Top Features by Importance:")
print(importance_df.head(10))  # Show the top 10 features

# Plot the feature importance
plt.figure(figsize=(12, 8))
plt.barh(importance_df["Feature"], importance_df["Importance"], color="skyblue")
plt.gca().invert_yaxis()  # Invert y-axis to show the most important feature at the top
plt.title("Feature Importance", fontsize=14)
plt.xlabel("Importance", fontsize=12)
plt.ylabel("Features", fontsize=12)
plt.grid(alpha=0.4)
plt.show()


In [ ]:
import joblib

# Save the trained model to a file with versioning
model_file = "random_forest_model_v1.pkl"
joblib.dump(model, model_file)

print(f"Model has been saved as '{model_file}'.")



In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters grid
param_grid = {
    'n_estimators': [50, 100, 200],        # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],      # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],      # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],        # Minimum samples required to be at a leaf node
}

# Initialize the GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    scoring='r2',  # Use R-squared as the evaluation metric
    cv=3,          # 3-fold cross-validation
    verbose=2,
    n_jobs=-1      # Use all available processors
)

# Perform the grid search on the training data
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("\nHyperparameter Tuning Completed!")
print(f"Best Parameters: {best_params}")
print(f"Best R-squared Score on Training Data: {grid_search.best_score_}")


In [ ]:
# Predict on the test set using the tuned model
y_pred_tuned = best_model.predict(X_test)

# Evaluate the tuned model
mse_tuned = mean_squared_error(y_test, y_pred_tuned)  # Mean Squared Error
r2_tuned = r2_score(y_test, y_pred_tuned)  # R-squared Score

# Display the evaluation results for the tuned model
print("Tuned Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse_tuned}")
print(f"R-squared (R2 Score): {r2_tuned}")


In [ ]:
# Display a comparison of the base and tuned models
print("\nComparison of Base Model vs Tuned Model:")
print(f"Base Model R2 Score: {r2}")
print(f"Tuned Model R2 Score: {r2_tuned}")


In [ ]:
# Save the tuned model as version 2
tuned_model_file = "random_forest_model_v2.pkl"
joblib.dump(best_model, tuned_model_file)

print(f"Tuned model has been saved as '{tuned_model_file}'.")


In [ ]:
# Define a threshold for low importance
importance_threshold = 0.01  # Features with importance < 0.01 will be removed

# Filter features with importance >= threshold
important_features = importance_df[importance_df["Importance"] >= importance_threshold]["Feature"].tolist()

# Create a reduced feature set
X_train_reduced = X_train[important_features]
X_test_reduced = X_test[important_features]

print(f"Number of important features retained: {len(important_features)}")
print(f"Number of features removed: {len(X_train.columns) - len(important_features)}")


In [ ]:
# Train a new Random Forest model using reduced features
reduced_model = RandomForestRegressor(random_state=42)
reduced_model.fit(X_train_reduced, y_train)

# Evaluate the reduced model
y_pred_reduced = reduced_model.predict(X_test_reduced)

mse_reduced = mean_squared_error(y_test, y_pred_reduced)
r2_reduced = r2_score(y_test, y_pred_reduced)

print("\nReduced Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse_reduced}")
print(f"R-squared (R2 Score): {r2_reduced}")


In [ ]:
print("\nComparison of All Models:")
print(f"Base Model R2 Score: {r2}")
print(f"Tuned Model R2 Score: {r2_tuned}")
print(f"Reduced Model R2 Score: {r2_reduced}")
